In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def conv_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, kernel_size=3, padding=1))

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concatenate input and output of each block along the channels
            X = torch.cat((X, Y), dim=1)
        return X

In [4]:
blk = DenseBlock(2, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 23, 8, 8])

In [5]:
def transition_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

In [6]:
blk = transition_block(10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

In [7]:
class DenseNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [8]:
@d2l.add_to_class(DenseNet)
def __init__(self, num_channels=64, growth_rate=32, arch=(4, 4, 4, 4),
             lr=0.1, num_classes=10):
    super(DenseNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, num_convs in enumerate(arch):
        self.net.add_module(f'dense_blk{i+1}', DenseBlock(num_convs,
                                                          growth_rate))
        # The number of output channels in the previous dense block
        num_channels += num_convs * growth_rate
        # A transition layer that halves the number of channels is added
        # between the dense blocks
        if i != len(arch) - 1:
            num_channels //= 2
            self.net.add_module(f'tran_blk{i+1}', transition_block(
                num_channels))
    self.net.add_module('last', nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

In [9]:
model = DenseNet(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
trainer.fit(model, data)

RuntimeError: GET was unable to find an engine to execute this computation

In [10]:
''' 
1. 为什么在过渡层中使用平均池化而不是最大池化？
   
   平均池化在过渡层中的使用是为了减小特征图的尺寸，同时保留更多的空间信息。相比之下，最大池化只保留最显著的特征，可能会导致一些空间信息的丢失。此外，平均池化对输入的响应更平滑，有助于降低过拟合的风险。

2. 在DenseNet论文中提到的优点之一是其模型参数比ResNet的参数少。为什么会这样？

   这是因为DenseNet的设计允许特征重用，从而减少了每个层所需的参数数量。在DenseNet中，每个层都直接连接到其之前的所有层，这意味着网络可以利用之前层的特征而无需重新计算。因此，DenseNet可以使用较少的参数来实现与ResNet相当的性能。

3. DenseNet被批评的一个问题是其高内存消耗。这真的是这样吗？尝试将输入形状更改为(224, 224)以经验性地比较实际GPU内存消耗。

   是的，DenseNet的内存消耗相对较高，因为它在每个层之间保持密集连接，这导致了较大的特征图。当输入尺寸增加时，内存消耗可能会变得更明显。为了经验性地比较内存消耗，您可以尝试使用不同的输入尺寸并监控GPU内存使用情况。

4. 您能想到一种替代方法来减少内存消耗吗？您需要如何更改框架？

   减少内存消耗的一种方法是使用更小的增长率（k值），这将导致每个层生成较少的特征图。但是，这可能会影响模型的性能。另一种方法是使用稀疏连接而不是密集连接。这可以减少特征图的数量，从而降低内存消耗。为了实现这种方法，您需要更改DenseNet的框架，使每个层只连接到部分先前层，而不是所有先前层。

5. 实现DenseNet论文（Huang等人，2017）表1中介绍的各种DenseNet版本。

   要实现DenseNet论文中的不同版本，您需要根据表1中的建议调整网络的深度、增长率和过渡层。具体来说，您可以更改DenseNet类的参数，例如设置不同的层数（L）、增长率（k）和压缩率（θ）。

6. 通过应用DenseNet思想设计一个基于MLP的模型。将其应用于第5.7节中的房价预测任务。

   要将DenseNet应用于MLP，您可以创建一个具有密集连接的多层感知器。在每个隐藏层之间，您可以添加一个连接层，将当前层的输出与之前所有层的输出连接起来。在实现这样的模型时，您需要确保输入和输出尺寸匹配房价预测任务的要求。然后，您可以使用类似于第5.7节中的方法来训练和评估模型。
   ''' 

' \n1. 为什么在过渡层中使用平均池化而不是最大池化？\n   \n   平均池化在过渡层中的使用是为了减小特征图的尺寸，同时保留更多的空间信息。相比之下，最大池化只保留最显著的特征，可能会导致一些空间信息的丢失。此外，平均池化对输入的响应更平滑，有助于降低过拟合的风险。\n\n2. 在DenseNet论文中提到的优点之一是其模型参数比ResNet的参数少。为什么会这样？\n\n   这是因为DenseNet的设计允许特征重用，从而减少了每个层所需的参数数量。在DenseNet中，每个层都直接连接到其之前的所有层，这意味着网络可以利用之前层的特征而无需重新计算。因此，DenseNet可以使用较少的参数来实现与ResNet相当的性能。\n\n3. DenseNet被批评的一个问题是其高内存消耗。这真的是这样吗？尝试将输入形状更改为(224, 224)以经验性地比较实际GPU内存消耗。\n\n   是的，DenseNet的内存消耗相对较高，因为它在每个层之间保持密集连接，这导致了较大的特征图。当输入尺寸增加时，内存消耗可能会变得更明显。为了经验性地比较内存消耗，您可以尝试使用不同的输入尺寸并监控GPU内存使用情况。\n\n4. 您能想到一种替代方法来减少内存消耗吗？您需要如何更改框架？\n\n   减少内存消耗的一种方法是使用更小的增长率（k值），这将导致每个层生成较少的特征图。但是，这可能会影响模型的性能。另一种方法是使用稀疏连接而不是密集连接。这可以减少特征图的数量，从而降低内存消耗。为了实现这种方法，您需要更改DenseNet的框架，使每个层只连接到部分先前层，而不是所有先前层。\n\n5. 实现DenseNet论文（Huang等人，2017）表1中介绍的各种DenseNet版本。\n\n   要实现DenseNet论文中的不同版本，您需要根据表1中的建议调整网络的深度、增长率和过渡层。具体来说，您可以更改DenseNet类的参数，例如设置不同的层数（L）、增长率（k）和压缩率（θ）。\n\n6. 通过应用DenseNet思想设计一个基于MLP的模型。将其应用于第5.7节中的房价预测任务。\n\n   要将DenseNet应用于MLP，您可以创建一个具有密集连接的多层感知器。在每个隐藏层之间，您可以添加一个连接层，将当前层的输出与之前所有层的输出连接起来。在实